In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
import urllib.parse
from thefuzz import fuzz

In [ ]:
# Define API parameters

url = "https://api.semanticscholar.org/graph/v1/paper/search"

# Insert your API key here
api_key = ""

headers = {
    "x-api-key": api_key  
}

In [1]:
# Send a GET request to the CLEF initiative website and parse the HTML response
resp = requests.get('https://www.clef-initiative.eu/')
soup = BeautifulSoup(resp.text,features="html.parser")

# Find all the div elements that contain the links to the publisher pages
links = soup.findAll("div", attrs={"class": "row portfolio-container"})

# Initialize lists to store DOI and CEUR links separately
publisher_link_list_doi = []
publisher_link_list_ceur = []

#if not os.path.exists(folder_location):os.mkdir(folder_location)
#https://stackoverflow.com/questions/72827238/how-to-download-all-pdf-files-from-multiple-urls-python

# Loop through each set of a-tags in the links div
for a_tags in links:
    
    # Find all anchor tags with the title "Publisher Page"
    publisher_links = a_tags.findAll("a", attrs={"title": "Publisher Page"})
    for link in publisher_links:
        # Check if the link is a DOI link and append it to the respective list
        if "doi.org" in link["href"]:
            publisher_link_list_doi.append(link["href"])
        else:
            # Otherwise, treat it as a CEUR link
            publisher_link_list_ceur.append(link["href"])

# Remove duplicates from the CEUR link list using set conversion
publisher_link_list_ceur = list(set(publisher_link_list_ceur))

# Remove a specific CEUR link from the list, if not needed
publisher_link_list_ceur.remove("https://ceur-ws.org/Vol-3497/")



['https://ceur-ws.org/Vol-3497/', 'https://ceur-ws.org/Vol-3497/', 'http://ceur-ws.org/Vol-3180/', 'http://ceur-ws.org/Vol-3180/', 'http://ceur-ws.org/Vol-2936/', 'http://ceur-ws.org/Vol-2936/', 'http://ceur-ws.org/Vol-2696/', 'http://ceur-ws.org/Vol-2696/', 'http://ceur-ws.org/Vol-2380/', 'http://ceur-ws.org/Vol-2380/', 'http://ceur-ws.org/Vol-2125/', 'http://ceur-ws.org/Vol-2125/', 'http://ceur-ws.org/Vol-1866/', 'http://ceur-ws.org/Vol-1866/', 'http://ceur-ws.org/Vol-1609/', 'http://ceur-ws.org/Vol-1609/', 'http://ceur-ws.org/Vol-1391/', 'http://ceur-ws.org/Vol-1391/', 'http://ceur-ws.org/Vol-1180/', 'http://ceur-ws.org/Vol-1180/', 'http://ceur-ws.org/Vol-1179/', 'http://ceur-ws.org/Vol-1179/', 'http://ceur-ws.org/Vol-1178/', 'http://ceur-ws.org/Vol-1178/', 'http://ceur-ws.org/Vol-1177/', 'http://ceur-ws.org/Vol-1177/', 'http://ceur-ws.org/Vol-1176/', 'http://ceur-ws.org/Vol-1176/', 'http://ceur-ws.org/Vol-1175/', 'http://ceur-ws.org/Vol-1175/', 'http://ceur-ws.org/Vol-1174/', 'http

In [2]:
document_metadata_list = []

# Iterate over each link in the CEUR publisher list
for i in publisher_link_list_ceur:
    
    # Send a GET request to the CEUR page and parse the HTML response
    resp = requests.get(i)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    # Find all ul elements on the page (usually containing lists of papers)
    ul_elements = soup.find_all('ul')
    
    # Extract the publication year from the page
    year = soup.find("span", attrs={"class" : "CEURVOLACRONYM"})
   
    # Regular expression to find a 4-digit year
    pattern = r'\b\d{4}\b'
    year = re.search(pattern, year.text)

    # Regular expression to find the volume number
    pattern = r'Vol-\d{4}'
    
    # Extract the full title of the volume
    full_title = soup.find("span", attrs={"class": "CEURFULLTITLE"})
    
    # Find the volume number in the URL
    volume = re.findall(pattern, i)
    
    # Iterate over each ul element found
    for ul in ul_elements:
        # Extract information from each li element within the ul
        li_elements = ul.find_all('li')
        
        # Loop through each list item (li) in the ul element
        for li in li_elements:
    
            # Extract the text content of the span element with class "CEURTITLE" (paper title)
            title_span = li.find('span', class_='CEURTITLE')
            title = title_span.text.strip() if title_span else ""
            title = re.sub("\n\s+", " ", title)
            
            # Skip if the title is "Introduction", "Preface", "Foreword", or if title_span is None
            if title in ["Introduction", "Preface", "Foreword"] or title_span == None:
                continue  # Skip the current li element

            
            document_info = []
    
            # Extract the href attribute of the a tag (link to the paper)
            href = li.find('a')['href']
    
            # Extract the text content of all span elements with class "CEURAUTHOR" (authors)
            authors = li.find_all('span', class_='CEURAUTHOR')
            author_list = [author.text for author in authors]  # Create a list of author names
            
            # Special case for years 2000 to 2015
            if year.group(0) in ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']:
                
                # Extract the author names from a single span element (older format)
                author_span = li.find("span", attrs={"class" : "CEURAUTHORS"})
                authors = author_span.text.strip() if author_span else ""

                # Split the author names based on commas and strip any extra whitespace
                author_list = [author.strip() for author in authors.split(',')]

            # Find the previous h3 element (likely containing session or section title)
            previous_h3 = li.find_previous('span', attrs={"class" :'CEURSESSION'})

            # Construct the full URL to the PDF file
            url = "https://ceur-ws.org/" + volume[0] + "/" + href
            
            # Append the extracted data to the list for this document
            document_info.extend([year.group(0), full_title.text, volume[0], href, title, previous_h3.text, author_list, url])
            
            # Add the document's information to the main list
            document_metadata_list.append(document_info)
    
   

if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
if
i

In [3]:
columns = ["PubYear", "CEUR Title", "Volume", "filename", "Title", "Section", "Authors", "url"]

ceur_proceedings_df = pd.DataFrame(document_metadata_list, columns=columns)


## Filter unwanted documents

In [5]:
# Remove CEUR proceeding documents from 2023

ceur_proceedings_df  = ceur_proceedings_df [ceur_proceedings_df ["PubYear"] != "2023"]

In [7]:
# Define list of unwanted titles

titles_to_remove = ["Foreword", "Introduction", "Contents", "Appendix A: Results of the Monolingual. Bilingual, Multilingual and Domain-Specific Tracks: Introduction",
                   "Appendix A: Results of the Monolingual. Bilingual, Multilingual and Domain-Specific Tracks: Run Statistics",
                   "Appendix B: Results of the Multiple Language Question Answering Track", "Appendix C: Results of the Cross-Language Image Retrieval Track",
                   "What Happened in CLEF 2005", "Appendix A: Results of the Core Tracks and Domain-Specific Tracks", "Appendix B: Results of the GeoCLEF Tracks","Appendix C: Cross Language Speech Retrieval Results",
                   "Appendix D: Participating Institutions", "What Happened in CLEF 2006: Introduction to the Working Notes", "Scientific Data of an Evaluation Campaign: Do We Properly Deal With Them?",
                   "Appendix A: Results of the Ad-hoc Bilingual and Monolingual Tasks","Appendix B: Results of the Ad-hoc Robust Task","Appendix C: Results of the Domain-Specific Track","Appendix D: Results of the GeoCLEF Track",
                   "Appendix E: Statistics for the QA@CLEF Track","Appendix F: Participating Institutions","What happened in CLEF 2007: Introduction to the Working Notes","Appendix A: Results of the Ad-hoc Bilingual and Monolingual Tasks",
                   "Appendix B: Results of the Core Tracks - Ad-hoc Robust Tasks","Appendix C: Results of the Domain Specific Track", "Appendix D: Results of GeoCLEF Track","Appendix E: Participating Institutions",
                   "What happened in CLEF 2008: Introduction to the Working Notes", "Appendix A: Results of the TEL@CLEF Task","Appendix B: Results of the Persian@CLEF Task", "Appendix C: Results of the Robust Track","Appendix D: Results of Domain Specific Track", "Appendix E: Results of GeoCLEF Track", 
                    "Appendix F: Partecipating Institutions", "What Happened in CLEF2009: Introduction to the Working Notes", "Appendix A: Results of the TEL@CLEF Track","Appendix B: Results of the Persian@CLEF Track",
                   "Appendix C: Results of the Robust-WSD Track", "Appendix D: Results of the Grid@CLEF Track","Appendix E: CLEF 2009 Participating Institutions","Introduction to the CLEF 2010 labs", "Introduction to the CLEF 2011 Labs",
                   "Introduction to the CLEF 2012 Labs", "Introduction to the CLEF2013 Labs and Workshop", "Preface", "Author index" ]

In [8]:
# Remove unwanted titles

ceur_proceedings_df  = ceur_proceedings_df [~ceur_proceedings_df ['Title'].isin(titles_to_remove)]

In [10]:
# Read in Dataframe with Metadata and assigned CEUR IDs

df_CEUR_id = pd.read_parquet("../../../data/metadata_CEUR_instititution_missing_with_ID.parquet")

In [12]:
# Merge the two dataframes two assign CEUR IDs

ceur_proceedings_df_with_ID = pd.merge(ceur_proceedings_df, df_CEUR_id, on=['PubYear', 'Title', 'CEUR Title', 'Volume', 'filename', 'Section'], how='left', suffixes=('', '_df2'))

In [13]:
ceur_proceedings_df_with_ID

,PubYear,CEUR Title,Volume,filename,Title,Section,Authors,url,Authors_df2,ID
0,2016,Working Notes of CLEF 2016 - Conference and La...,Vol-1609,16090001.pdf,Task 1 of the CLEF eHealth Evaluation Lab 2016...,CLEFeHealth,"[Hanna Suominen, Liyuan Zhou, Lorraine Goeurio...",https://ceur-ws.org/Vol-1609/16090001.pdf,"[Hanna Suominen, Liyuan Zhou, Lorraine Goeurio...",ceur_1019
1,2016,Working Notes of CLEF 2016 - Conference and La...,Vol-1609,16090015.pdf,The IR Task at the CLEF eHealth Evaluation Lab...,CLEFeHealth,"[Guido Zuccon, Joao Palotti, Lorraine Goeuriot...",https://ceur-ws.org/Vol-1609/16090015.pdf,"[Guido Zuccon, Joao Palotti, Lorraine Goeuriot...",ceur_1020
2,2016,Working Notes of CLEF 2016 - Conference and La...,Vol-1609,16090028.pdf,Clinical Information Extraction at the CLEF eH...,CLEFeHealth,"[Aurélie Névéol, Kevin Bretonnel Cohen, Cyril ...",https://ceur-ws.org/Vol-1609/16090028.pdf,"[Aurélie Névéol, Kevin Bretonnel Cohen, Cyril ...",ceur_1021
3,2016,Working Notes of CLEF 2016 - Conference and La...,Vol-1609,16090043.pdf,Comparison of Several Word embedding Sources f...,CLEFeHealth,"[Julie Budaher, Mohannad Almasri, Lorraine Goe...",https://ceur-ws.org/Vol-1609/16090043.pdf,"[Julie Budaher, Mohannad Almasri, Lorraine Goe...",ceur_1022
4,2016,Working Notes of CLEF 2016 - Conference and La...,Vol-1609,16090047.pdf,SIBM at CLEF eHealth Evaluation Lab 2016: Extr...,CLEFeHealth,"[Chloé Cabot, Lina F. Soualmia, Badisse Dahamn...",https://ceur-ws.org/Vol-1609/16090047.pdf,"[Chloé Cabot, Lina F. Soualmia, Badisse Dahamn...",ceur_1023
...,...,...,...,...,...,...,...,...,...,...
3212,2022,Proceedings of the Working Notes of CLEF 2022 ...,Vol-3180,paper-261.pdf,Quality-Aware Argument Re-Ranking for Comparat...,Touché: Argument Retrieval,"[Niclas Arnhold, Philipp Rösner, Tobias Xylander]",https://ceur-ws.org/Vol-3180/paper-261.pdf,"[Niclas Arnhold, Philipp Rösner, Tobias Xylander]",ceur_3328
3213,2022,Proceedings of the Working Notes of CLEF 2022 ...,Vol-3180,paper-262.pdf,The Pearl Retriever: Two-Stage Retrieval for P...,Touché: Argument Retrieval,"[Sebastian Schmidt, Jonas Probst, Bianca Barte...",https://ceur-ws.org/Vol-3180/paper-262.pdf,"[Sebastian Schmidt, Jonas Probst, Bianca Barte...",ceur_3329
3214,2022,Proceedings of the Working Notes of CLEF 2022 ...,Vol-3180,paper-263.pdf,Touché - Task 1 - Team Korg: Finding pairs of ...,Touché: Argument Retrieval,"[Cuong Vo Ta, Florian Reiner, Immanuel von Det...",https://ceur-ws.org/Vol-3180/paper-263.pdf,"[Cuong Vo Ta, Florian Reiner, Immanuel von Det...",ceur_3330
3215,2022,Proceedings of the Working Notes of CLEF 2022 ...,Vol-3180,paper-264.pdf,Using BERT to retrieve relevant and argumentat...,Touché: Argument Retrieval,"[Nils Wenzlitschke, Pia Sülzle]",https://ceur-ws.org/Vol-3180/paper-264.pdf,"[Nils Wenzlitschke, Pia Sülzle]",ceur_3331


In [14]:
ceur_proceedings_df_with_ID.drop(columns=['Authors_df2'], inplace=True)


In [15]:
ceur_proceedings_df_with_ID.to_parquet("../../../data/metadata_CEUR.parquet")